In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import chainer
import gensim

In [2]:
cd ../

C:\Users\akite\OneDrive\デスクトップ\source code\twitter-compe


In [3]:
train_df = pd.read_csv('./data/preprosessing_train.csv')
test_df = pd.read_csv('./data/preprosessing_test.csv')

In [4]:
X_df_train = train_df[["is_url","is_location","num_emoji","num_reply","num_hash","text_length","num_kusa","is_date","is_time","text_wakati"]]
Y_df_train = train_df["flg"].astype(np.int64)
X_train =  X_df_train.values
y_train = Y_df_train.values
X_df_test =  test_df[["is_url","is_location","num_emoji","num_reply","num_hash","text_length","num_kusa","is_date","is_time","text_wakati"]]
X_test = X_df_test.values


In [5]:
#特徴量配列
X_train_feat = X_train[:,:8]
X_test_feat = X_test[:,:8]

In [6]:
#特徴量標準化
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
X_train_feat_ss = ss.fit_transform(X_train_feat)
X_test_feat_ss = ss.transform(X_test_feat)

In [7]:
#文字カラムに対する欠損値削除
X_train_del_one = X_train[:,9]
for idx in range(len(X_train_del_one)):
    if type(X_train_del_one[idx]) is float:
        X_train_del_one[idx] = " " 

X_test_del_one = X_test[:,9]
for idx in range(len(X_test_del_one)):
    if type(X_test_del_one[idx]) is float:
        X_test_del_one[idx] = " " 

In [8]:
#単語ベクトル化(最大値、最小値を設定した)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = TfidfVectorizer(min_df=2/16228, max_df=16220/16228)
X_train_cv = cv.fit_transform(X_train_del_one)
X_test_cv = cv.transform(X_test_del_one)

In [9]:
#文字配列と特徴量の配列を結合する
X_train = np.concatenate([X_train_cv.toarray(),X_train_feat_ss],1)
X_test = np.concatenate([X_test_cv.toarray(),X_test_feat_ss],1)

In [10]:
#型変換
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [11]:
#スパース配列に変換
from scipy.sparse import csr_matrix
X_train_csr = csr_matrix(X_train)
X_test_csr = csr_matrix(X_test)

In [12]:
#XGB
import xgboost as xgb
model = xgb.XGBClassifier(learning_rate=1.0,
                         colsample_bytree = 1.0,
                          max_depth = 10,
                          subsample = 1
                         )
model.fit(X_train_csr, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=1.0, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [13]:
y_pred = model.predict(X_test_csr)

In [14]:
sum(y_pred)

826

In [15]:
write_df = pd.DataFrame()
write_df["flg"] = y_pred
write_df.to_csv('./data/xgboost.csv', index=False)